In [107]:
import math 

def discretize(waiting_times):
    for waiting_time in waiting_times:
        if waiting_time > 99.9:
            waiting_time = 99
    return 10000000* abs(round(waiting_times[0])) + 10000*abs(round(waiting_times[1])) + 10*abs(round(waiting_times[2]))

In [113]:
import gym
from gym import spaces
import numpy as np
from math import ceil

class TrafficGenerator(gym.Env):
    
    #doesnt really workj
    def getObs(self):
        # Loop over the packet types
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
           
            observation.append([current_length, current_waiting_time])

       
        observation = np.array(observation)
        return observation
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(3)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        # Queue maximum length set to 100
        self.observation_space = spaces.Discrete(1000000000)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.3, 0.25, 0.4]
        self.mean_delay_req = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.curr_mean_delay_best_effort = 0
        self.packet = 1
        self.timeslot = 1
        self.totaltime = self.timeslot
        
        # Initialize the queues
        self.queues = [[], [], []]
      
    
    def step(self, action):  
        self.totaltime += self.timeslot
        print(self.totaltime)
        #packet generator
        for i in range(len(self.packetInfo)):
            for sublist in self.packetInfo:
                if sublist[0] == i and np.random.uniform() < sublist[1]:
                    self.queues[i].append(self.packet)
                    #print("Appending to queue", i)
        
        for i in range(len(self.queues)):
            for j in range(len(self.queues[i])):
                self.queues[i][j] -= self.timeslot
                
        
        # services the queues
        #for i in range(len(self.queues)):
            #print("queue ", i, ": has a length of", len(self.queues[i]))
        #print("ACTION queue ", action, ": has a length of", len(self.queues[action]))
        if action < 3 and len(self.queues[action]) > 0:
            self.queues[action].pop(0)
            for i in range(len(self.queues[action])):
                self.queues[action][i] -= 1
        

        
        #TODO rewardoptions?
        reward = 1
        
        observation = []
        waiting_times = []
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            current_waiting_time = np.round(current_waiting_time, 1)
            observation.append([current_length, current_waiting_time])
            waiting_times.append(current_waiting_time)
            if i == 2:
                #reward option
                if self.curr_mean_delay_best_effort == 0:
                    curr_mean_delay_best_effort = current_waiting_time
                if self.curr_mean_delay_best_effort < current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = 1
                elif self.curr_mean_delay_best_effort > current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = -1
                else:
                    reward = 1
        
                #print(self.curr_mean_delay_best_effort)
                #print(current_waiting_time)
        print(f"observation{observation}")
        if observation[0][1] < -4:
            reward = -10
        elif observation[1][1] < -6:
            reward = -10
        observation = np.array(observation)
        
         # Loop over the packet types
        """
        observation = []
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
    
            observation.append([current_length, current_waiting_time])

        observation = np.array(observation)
        """
        info = {"mean_delay_0": self.mean_delay_req[0],
        "mean_delay_1": self.mean_delay_req[1],
        "mean_delay_2": self.mean_delay_req[2],
        "arrival_rate_0": self.arrival_rate[0],
        "arrival_rate_1": self.arrival_rate[1],
        "arrival_rate_2": self.arrival_rate[2]}
        done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0 or (self.totaltime >= 100)
        
        waiting_times = discretize(waiting_times)
        
        return waiting_times, reward, done, info
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        self.totaltime = 1
        return [0,0,0]
        
    def render(self):
        pass

In [114]:
env = TrafficGenerator()
print('State space: ', env.observation_space)
print('Action space: ', env.action_space)


State space:  Discrete(1000000000)
Action space:  Discrete(3)


In [115]:
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
while x < 100:
    print("~~~~~~~~~")
    action = env.action_space.sample()
    
    print("chosen", action)
    obs,reward, done, info = env.step(action)
    print(obs)
    print("reward", reward)
    
    x = x +1

~~~~~~~~~
chosen 1
2
observation[[0, 0.0], [0, 0.0], [1, 0.0]]
0
reward 1
~~~~~~~~~
chosen 2
3
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
0
reward 1
~~~~~~~~~
chosen 2
4
observation[[2, -0.5], [1, 0.0], [0, 0.0]]
0
reward 1
~~~~~~~~~
chosen 2
5
observation[[3, -1.0], [1, -1.0], [0, 0.0]]
10010000
reward 1
~~~~~~~~~
chosen 2
6
observation[[3, -2.0], [2, -1.0], [0, 0.0]]
20010000
reward 1
~~~~~~~~~
chosen 2
7
observation[[3, -3.0], [2, -2.0], [0, 0.0]]
30020000
reward 1
~~~~~~~~~
chosen 0
8
observation[[2, -4.5], [3, -2.0], [0, 0.0]]
40020000
reward -10
~~~~~~~~~
chosen 0
9
observation[[2, -3.5], [3, -3.0], [1, 0.0]]
40030000
reward 1
~~~~~~~~~
chosen 0
10
observation[[1, -3.0], [4, -3.0], [1, -1.0]]
30030010
reward -1
~~~~~~~~~
chosen 2
11
observation[[2, -2.0], [4, -4.0], [0, 0.0]]
20040000
reward 1
~~~~~~~~~
chosen 1
12
observation[[3, -2.0], [4, -4.0], [0, 0.0]]
20040000
reward 1
~~~~~~~~~
chosen 0
13
observation[[2, -2.5], [5, -4.0], [0, 0.0]]
20040000
reward 1
~~~~~~~~~
chosen 2
14


In [116]:
import random 

action_size = env.action_space.n
state_size = env.observation_space.n
qtable = np.zeros((state_size, action_size))

# Hyperparameters
total_episodes = 500
total_test_episodes = 10
max_steps = 99
learning_rate = 0.7
gamma = 0.618
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

# Train
for episode in range(total_episodes):
    state = env.reset()
    
    for step in range(max_steps):
        exp_exp_tradeoff = random.uniform(0, 1)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state, :])
        else:
            action = env.action_space.sample()

        new_state, reward, done, info = env.step(action)
        new_state = int(new_state)
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state]) - qtable[state, action])
        
        state = new_state
        if done: break

    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * (episode+1))


# Play the Game
rewards = []

for episode in range(total_test_episodes):
    total_rewards = 0
    total_penalties = 0
    state = env.reset()
    print('='*20)
    print("[*] Episode", episode)
    print('='*20)

    for step in range(max_steps):
        env.render()
        action = np.argmax(qtable[state, :])
        state, reward, done, info = env.step(action)
#         print(reward)
        if reward == -10:
            penlties += 1
        else:
            total_rewards += reward
        print("total time: {}", env.totaltime)
        if done:
            rewards.append(total_rewards)
            print('[*] Score', total_rewards)
            break

env.close()
print('[*] Average Score: ' + str(sum(rewards) / total_test_episodes))

2
observation[[0, 0.0], [1, 0.0], [1, 0.0]]
3
observation[[0, 0.0], [1, -1.0], [2, -0.5]]
4
observation[[0, 0.0], [1, -2.0], [2, -1.5]]
5
observation[[0, 0.0], [2, -1.5], [2, -2.0]]
6
observation[[0, 0.0], [3, -1.7], [3, -2.0]]
7
observation[[0, 0.0], [4, -2.0], [3, -2.3]]
8
observation[[0, 0.0], [4, -3.0], [3, -2.7]]
9
observation[[0, 0.0], [4, -3.2], [3, -3.7]]
10
observation[[0, 0.0], [5, -3.4], [3, -4.7]]
11
observation[[0, 0.0], [4, -4.8], [3, -5.7]]
12
observation[[0, 0.0], [3, -6.0], [3, -6.7]]
13
observation[[1, 0.0], [3, -7.0], [2, -8.0]]
14
observation[[0, 0.0], [3, -8.0], [2, -9.0]]
15
observation[[1, 0.0], [2, -9.0], [2, -10.0]]
16
observation[[1, -1.0], [2, -10.0], [2, -6.0]]
17
observation[[1, -2.0], [2, -6.0], [2, -7.0]]
18
observation[[0, 0.0], [2, -7.0], [2, -8.0]]
19
observation[[0, 0.0], [1, -4.0], [3, -6.0]]
20
observation[[1, 0.0], [1, -5.0], [3, -3.0]]
21
observation[[1, -1.0], [1, -6.0], [2, -3.5]]
22
observation[[1, -2.0], [1, -7.0], [2, -3.0]]
23
observation[[2

2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [1, 0.0], [0, 0.0]]
3
observation[[0, 0.0], [1, -1.0], [0, 0.0]]
4
observation[[0, 0.0], [1, -2.0], [1, 0.0]]
5
observation[[0, 0.0], [0, 0.0], [2, -0.5]]
6
observation[[0, 0.0], [0, 0.0], [1, -2.0]]
7
observation[[0, 0.0], [0, 0.0], [1, -1.0]]
8
observation[[0, 0.0], [0, 0.0], [1, -1.0]]
9
observation[[0, 0.0], [0, 0.0], [2, -1.0]]
10
observation[[0, 0.0], [0, 0.0], [2, -2.0]]
11
observation[[0, 0.0], [0, 0.0], [2, -2.0]]
12
observation[[1, 0.0], [1, 0.0], [1, -3.0]]
13
observation[[1, -1.0], [1, -1.0], [1, -4.0]]
14
observation[[1, -2.0], [1, -2.0], [0, 0.0]]
15
observation[[1, -3.0], [0, 0.0], [1, 0.0]]
16
observation[[1, -4.0], [0, 0.0], [1, -1.0]]
17
observation[[1, -5.0], [0, 0.0], [2, -1.0]]
18
observation[[1, -6.0], [0, 0.0], [3, -1.3]]
19
observation[[1, -7.0], [0, 0.0], [3, -2.3]]
20
observation[[1, -8.0], [0, 0.0], [2, -3

3
observation[[0, 0.0], [0, 0.0], [2, -0.5]]
4
observation[[0, 0.0], [0, 0.0], [3, -1.0]]
5
observation[[0, 0.0], [1, 0.0], [3, -2.0]]
6
observation[[0, 0.0], [2, -0.5], [2, -3.5]]
7
observation[[0, 0.0], [2, -1.5], [1, -5.0]]
8
observation[[0, 0.0], [2, -2.0], [1, -6.0]]
9
observation[[0, 0.0], [3, -2.0], [1, -1.0]]
10
observation[[0, 0.0], [3, -3.0], [0, 0.0]]
11
observation[[0, 0.0], [3, -4.0], [0, 0.0]]
12
observation[[0, 0.0], [3, -5.0], [0, 0.0]]
13
observation[[1, 0.0], [4, -4.5], [0, 0.0]]
14
observation[[1, -1.0], [4, -5.5], [0, 0.0]]
15
observation[[1, -2.0], [4, -6.5], [0, 0.0]]
16
observation[[0, 0.0], [5, -6.0], [0, 0.0]]
17
observation[[0, 0.0], [5, -7.0], [0, 0.0]]
18
observation[[0, 0.0], [4, -7.8], [1, 0.0]]
19
observation[[0, 0.0], [3, -8.3], [2, -0.5]]
20
observation[[1, 0.0], [4, -7.0], [2, -1.5]]
21
observation[[1, -1.0], [3, -7.0], [2, -2.5]]
22
observation[[2, -1.0], [3, -8.0], [1, -4.0]]
23
observation[[3, -1.3], [2, -8.0], [2, -2.5]]
24
observation[[4, -1.8], [

63
observation[[0, 0.0], [3, -5.0], [9, -13.2]]
64
observation[[0, 0.0], [3, -4.0], [9, -14.2]]
65
observation[[0, 0.0], [3, -2.7], [9, -15.2]]
66
observation[[0, 0.0], [4, -2.8], [8, -16.0]]
67
observation[[0, 0.0], [5, -3.0], [7, -16.7]]
68
observation[[0, 0.0], [4, -4.2], [8, -15.5]]
69
observation[[1, 0.0], [3, -5.3], [8, -16.5]]
70
observation[[0, 0.0], [3, -6.3], [8, -17.5]]
71
observation[[0, 0.0], [3, -7.3], [9, -16.4]]
72
observation[[0, 0.0], [2, -8.5], [10, -15.7]]
73
observation[[0, 0.0], [1, -10.0], [11, -15.2]]
74
observation[[0, 0.0], [2, -5.5], [11, -16.2]]
75
observation[[1, 0.0], [1, -2.0], [12, -15.8]]
76
observation[[1, -1.0], [1, -3.0], [11, -16.1]]
77
observation[[1, -2.0], [0, 0.0], [12, -15.7]]
78
observation[[1, -3.0], [0, 0.0], [12, -16.7]]
79
observation[[1, -4.0], [0, 0.0], [11, -16.8]]
80
observation[[0, 0.0], [1, 0.0], [12, -16.3]]
81
observation[[0, 0.0], [2, -0.5], [12, -17.3]]
82
observation[[0, 0.0], [1, -2.0], [12, -18.3]]
83
observation[[1, 0.0], [0,

2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
3
observation[[2, -0.5], [0, 0.0], [0, 0.0]]
4
observation[[3, -1.0], [1, 0.0], [0, 0.0]]
5
observation[[3, -2.0], [1, -1.0], [0, 0.0]]
6
observation[[4, -2.2], [1, -2.0], [0, 0.0]]
7
observation[[4, -3.2], [1, -3.0], [0, 0.0]]
8
observation[[4, -4.2], [0, 0.0], [0, 0.0]]
9
observation[[5, -4.2], [0, 0.0], [1, 0.0]]
10
observation[[5, -4.6], [0, 0.0], [2, -0.5]]
11
observation[[4, -5.8], [1, 0.0], [2, -1.5]]
12
observation[[4, -5.2], [2, -0.5], [3, -1.7]]
13
observation[[4, -6.2], [2, -1.5], [2, -3.0]]
14
observation[[3, -7.0], [3, -1.7], [3, -2.7]]
15
observation[[3, -8.0], [2, -3.0], [3, -3.7]]


69
observation[[0, 0.0], [1, -1.0], [17, -16.1]]
70
observation[[0, 0.0], [1, -2.0], [18, -16.1]]
71
observation[[0, 0.0], [1, -1.0], [19, -16.2]]
72
observation[[0, 0.0], [1, -2.0], [19, -17.2]]
73
observation[[0, 0.0], [0, 0.0], [19, -18.2]]
74
observation[[1, 0.0], [0, 0.0], [18, -19.2]]
75
observation[[1, -1.0], [0, 0.0], [18, -19.1]]
76
observation[[1, -1.0], [1, 0.0], [18, -20.1]]
77
observation[[1, -2.0], [1, -1.0], [17, -21.1]]
78
observation[[1, -3.0], [2, -1.0], [16, -22.0]]
79
observation[[0, 0.0], [2, -2.0], [17, -21.6]]
80
observation[[1, 0.0], [1, -3.0], [17, -22.6]]
81
observation[[2, -0.5], [1, -4.0], [16, -23.5]]
82
observation[[2, -1.5], [2, -2.5], [16, -22.9]]
83
observation[[2, -2.5], [3, -2.3], [15, -23.6]]
84
observation[[2, -3.5], [3, -2.0], [15, -24.6]]
85
observation[[1, -5.0], [3, -3.0], [15, -25.6]]
86
observation[[1, -1.0], [3, -4.0], [15, -26.6]]
87
observation[[0, 0.0], [3, -5.0], [16, -25.9]]
88
observation[[0, 0.0], [3, -6.0], [16, -26.9]]
89
observation

50
observation[[1, -1.0], [1, -5.0], [6, -5.2]]
51
observation[[0, 0.0], [1, -6.0], [6, -6.2]]
52
observation[[1, 0.0], [1, -1.0], [6, -7.2]]
53
observation[[1, -1.0], [1, -2.0], [5, -8.0]]
54
observation[[1, -2.0], [1, -3.0], [5, -7.2]]
55
observation[[1, -3.0], [1, -4.0], [4, -8.2]]
56
observation[[1, -4.0], [1, -5.0], [3, -9.0]]
57
observation[[1, -5.0], [1, -6.0], [3, -6.7]]
58
observation[[1, -6.0], [0, 0.0], [3, -7.7]]
59
observation[[0, 0.0], [0, 0.0], [3, -8.7]]
60
observation[[0, 0.0], [0, 0.0], [4, -7.2]]
61
observation[[0, 0.0], [0, 0.0], [3, -6.7]]
62
observation[[0, 0.0], [0, 0.0], [3, -4.3]]
63
observation[[0, 0.0], [0, 0.0], [2, -4.5]]
64
observation[[0, 0.0], [0, 0.0], [1, -5.0]]
65
observation[[0, 0.0], [1, 0.0], [1, -1.0]]
66
observation[[1, 0.0], [1, -1.0], [0, 0.0]]
67
observation[[1, -1.0], [1, -2.0], [1, 0.0]]
68
observation[[2, -1.0], [1, -3.0], [0, 0.0]]
69
observation[[3, -1.3], [1, -4.0], [0, 0.0]]
70
observation[[3, -2.3], [0, 0.0], [0, 0.0]]
71
observation[[

47
observation[[1, 0.0], [2, -3.5], [5, -7.8]]
48
observation[[1, -1.0], [2, -4.5], [5, -7.2]]
49
observation[[1, -2.0], [3, -3.7], [4, -8.0]]
50
observation[[2, -1.5], [3, -4.7], [3, -8.3]]
51
observation[[1, -2.0], [4, -4.2], [4, -7.0]]
52
observation[[2, -1.5], [4, -5.2], [3, -6.7]]
53
observation[[1, -2.0], [4, -6.2], [3, -7.7]]
54
observation[[0, 0.0], [4, -7.2], [4, -6.5]]
55
observation[[0, 0.0], [4, -8.2], [3, -6.7]]
56
observation[[1, 0.0], [5, -7.4], [3, -4.3]]
57
observation[[0, 0.0], [5, -8.4], [4, -4.0]]
58
observation[[0, 0.0], [5, -9.4], [5, -4.0]]
59
observation[[0, 0.0], [6, -8.7], [5, -3.8]]
60
observation[[1, 0.0], [5, -9.0], [6, -4.0]]
61
observation[[0, 0.0], [5, -10.0], [6, -5.0]]
62
observation[[0, 0.0], [4, -10.2], [6, -6.0]]
63
observation[[0, 0.0], [4, -11.2], [6, -7.0]]
64
observation[[0, 0.0], [4, -12.2], [5, -8.0]]
65
observation[[1, 0.0], [5, -10.6], [5, -7.6]]
66
observation[[1, -1.0], [5, -11.6], [5, -7.2]]
67
observation[[0, 0.0], [6, -10.5], [5, -8.2]]

NameError: name 'penlties' is not defined